In [1]:
# import necessary packages
import os, sys
import glob
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from ipywidgets import FloatProgress
from PIL import Image
from scipy import ndimage
from importlib import reload

sys.path.append('../')

import utils.Preprocessing as Preprocessing

In [2]:
# options
image_path = "../pennycress_pods/test/test_images/"
mask_path = "../pennycress_pods/test/test_masks_preproc/"
image_save_path = "../pennycress_pods/test/test_images_by_pod/"
mask_save_path = "../pennycress_pods/test/test_masks_by_pod/"

plot = False

# load image data
image_names = glob.glob(mask_path + "*.png")
image_names = [os.path.basename(x) for x in image_names]

In [3]:
# split test masks
reload(Preprocessing)
Preprocessing.split_image(image_names=image_names,
                          image_path=image_path,
                          mask_path=mask_path,
                          image_save_path=image_save_path,
                          mask_save_path=mask_save_path,
                          plot=False)


Processing image:  1.tt8t x fc6-1.B.WBP-2022.W.png


  0%|          | 0/10 [00:00<?, ?it/s]

Processing image:  9_5.tt8t x fc5-1.B.WBP-2022.W_rep2.png


  0%|          | 0/11 [00:00<?, ?it/s]

Processing image:  10.tt8t x fc5-2.B.WBP-2022.W.png


...

Processing image:  550-ref-ISU_216-rep_1-W.png


  0%|          | 0/15 [00:00<?, ?it/s]

Processing image:  490-ref-ISU_073-rep_1-W.png


  0%|          | 0/15 [00:00<?, ?it/s]

Image Splitting Complete!


In [4]:
# options
image_path = "../pennycress_pods/train/train_images/"
mask_path = "../pennycress_pods/train/train_masks_preproc/"
image_save_path = "../pennycress_pods/train/train_images_by_pod/"
mask_save_path = "../pennycress_pods/train/train_masks_by_pod/"

plot = False

# load image data
image_names = glob.glob(mask_path + "*.png")
image_names = [os.path.basename(x) for x in image_names]

In [5]:
# split train masks
reload(Preprocessing)
Preprocessing.split_image(image_names=image_names,
                          image_path=image_path,
                          mask_path=mask_path,
                          image_save_path=image_save_path,
                          mask_save_path=mask_save_path,
                          plot=False)

Processing image:  33.479-single.B.field 2023.W.png


  0%|          | 0/10 [00:00<?, ?it/s]

Processing image:  557-ref-ISU_188-rep_3-W.png


  0%|          | 0/15 [00:00<?, ?it/s]

Processing image:  53.AFNM23_1.B.Wild 2023.S.png


...

Processing image:  521-ref-ISU_064-rep_1-W.png


  0%|          | 0/15 [00:00<?, ?it/s]

Processing image:  89.ISU386_B.B.ISU Wild.W.png


  0%|          | 0/10 [00:00<?, ?it/s]

Image Splitting Complete!
